In [1]:
from selenium import webdriver
from PIL import Image

from pyvirtualdisplay import Display
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd

from io import BytesIO

from selenium.webdriver.common.by import By

from pyvirtualdisplay import Display

from tqdm import tqdm



In [2]:
def highlight(driver, element, effect_time, color, border):
    """Highlights (blinks) a Selenium Webdriver element"""
    def apply_style(s):
        driver.execute_script("arguments[0].setAttribute('style', arguments[1]);",
                              element, s)
    original_style = element.get_attribute('style')
    apply_style(f"border: {border}px solid {color};")
    time.sleep(effect_time)

class Screenshotter():
    def __init__(self, save_dir):
        display = Display(visible=0, size=(1920, 1080))  
        display.start()

        self.service = Service('/home/veselovs/chromedriver')
        self.service.start()

        self.options = webdriver.ChromeOptions()
        self.driver = webdriver.Chrome(options = self.options)
        timeout = 25
        self.driver.set_page_load_timeout(timeout)
        
        self.save_dir = save_dir


    def extract_element(self, website_link, wiki):
        try:
            self.driver.get(website_link)
            time.sleep(3)
            element = self.driver.find_elements(By.XPATH, f'//a[contains(@href,"{wiki}")]')[0]
            parent = element.find_element(By.XPATH, "./..")
            self.driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'nearest'})", element)
            highlight(self.driver, element,5,"red",3)
            return element
        except:
            print("Didn't find element")
            parent = None
            return parent

    def take_screenshot(self,element, output_name):
        try:
            x = element.screenshot_as_png

            im = Image.open(BytesIO(x))
            im.save(self.save_dir + f'{output_name}.png')
#             self.driver.close()
            print("Saved image {}".format(output_name))
        except WebDriverException:
            print("Not wide enough")
            
        
    def iterate_over(self, url_links, wiki_links, output_names, element_screenshot = False):
        
        for url, wiki, output in tqdm(zip(url_links,wiki_links,output_names)):
            print(output)
            parent = self.extract_element(url, wiki)
            if parent != None:
                if element_screenshot == True:
                    self.take_screenshot(parent,output)
                else:
                    self.driver.save_screenshot(self.save_dir + f"{output}.png")
        self.driver.quit()

### Choose the subset of mentions we want to get links for. 

In [10]:
web_content = pd.read_csv("/scratch/venia/web2wiki/data/random_sample_html.csv").drop(columns = "Unnamed: 0")

In [ ]:
web_content = pd.read_csv("/scratch/venia/web2wiki/data/1st_order.csv").drop(columns = "Unnamed: 0")

In [17]:
web_content["url"].iloc[0]

'http://simon.butcher.name/archives/2007/06/10/Coming-to-America'

In [5]:
web_contentt = web_content

In [6]:
url_links = web_contentt["url"].values
wiki_links = web_contentt["wiki_url"].values
names = web_contentt.index.values

In [266]:
sc = Screenshotter()
sc.iterate_over(url_links, wiki_links,names)

# Add text on image

In [367]:
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw, ImageOps 

IMG_DIR = "/scratch/venia/web2wiki/data/screenshots/1000_sample/"

def process_image(im_file, row, save_dir):
    image = Image.open(IMG_DIR + im_file + ".png")
    image = ImageOps.expand(image, border=50, fill=(255,255,255))

    draw = ImageDraw.Draw(image) 

    # specified font size
    font = ImageFont.truetype('/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', 14) 

    text = f'url: {row["url"][0:80]}            \n parent 1: {row["a1"]}             parent 2: {row["a2"]}\
                     Wiki Title: {row["title"]}             Page rank: {row["#pr_pos"]}             Wiki links on URL: {row["num_wiki_on_url"]}'

    # drawing text size
    draw.text((0,0), text, font = font, align ="left",fill=(0,0,0)) 

    image.save(save_dir + im_file + "_clean.png")

In [341]:
import os
for (row_id, row) in web_contentt.iterrows():
    row_id = str(row_id)
    if os.path.exists(IMG_DIR + row_id + ".png"):
        process_image(row_id, row, IMG_DIR)


In [368]:
import os
IMG_DIR_no_wiki = "/scratch/venia/web2wiki/data/screenshots/1000_sample_no_wiki/"

for (row_id, row) in web_contentt.iterrows():
    row_id = str(row_id)
    if os.path.exists(IMG_DIR + row_id + ".png"):
        if not (("wiki" in row["url"]) | ("pedia" in row["url"])):
            process_image(row_id, row, IMG_DIR_no_wiki)


# Image categories

In [731]:
df = pd.read_parquet("/scratch/venia/web2wiki/data/web_content/iterative_coding_sample/tag_info.parquet")

In [732]:
df = df[df["wiki_links"] != "[]"]
for k in df.columns[1:]:
    df[k] = df[k].apply(lambda x: x[1:-1].split(", "))

cols = list(df.columns[1:])
df.drop_duplicates(subset = "url")


df.reset_index(drop = True,inplace =True)
df.index=df["url"]
df = df.drop(columns = "url")

In [761]:
df2 = pd.read_csv("/scratch/venia/web2wiki/data/tag_info_clean2.csv")

In [755]:
df2 = df2.sample(1000)

In [762]:
url_links = df2["url"]
wiki_links = df2["wiki_links"]
names = df2.index

In [ ]:
sc = Screenshotter(save_dir = "/scratch/venia/web2wiki/data/screenshots/1000_sample2/")
sc.iterate_over(url_links, wiki_links,names)

0it [00:00, ?it/s]

0


1it [00:11, 11.26s/it]

1


2it [00:21, 10.69s/it]

...

4


5it [00:48,  9.07s/it]

Didn't find element
5


6it [00:57,  9.22s/it]

6


In [767]:
df2["0th_order"] = df2[["is_tag_footer","is_tag_header", "is_class_footer","is_class_header","is_class_sidebar"]].sum(axis=1)
df2["1st_order_evidence"] = df2[["is_tag_sup","is_tag_cite"]].sum(axis=1)

In [770]:
df2.to_csv("/scratch/venia/web2wiki/data/sample_compare.csv",index=False)

In [ ]:
# CALCULATE THE DISTRIBUTION OVER 0TH ORDER 1ST ORDER 2ND ORDER

In [684]:
df2["is_more_than_40"] = df2["url_wiki_count"].apply(lambda x: 1 if x > 39 else 0)
df2["is_less_than_5"] = df2["url_wiki_count"].apply(lambda x: 1 if x < 5 else 0)


In [714]:
columns = ["is_blog","is_wiki","is_more_than_40","is_less_than_5","is_footer","is_header","is_tag_sup","is_class_comment","is_class_sidebar"]

df2["is_footer"] = df2.apply(lambda x: 1 if (x["is_tag_footer"] == 1) or (x["is_class_footer"] == 1) else 0,axis=1)
df2["is_header"] = df2.apply(lambda x: 1 if (x["is_tag_header"] == 1) or (x["is_class_header"] == 1) else 0,axis=1)


In [ ]:

#### folders
# wiki or pedia in url
# is blog
# more than 30 mentions 
# less than 10 mentions
# is_tag or class footer
# is tag_sup
# is tag_cite
# is class_sidebar
# is class_comment




In [715]:
for col in columns[4:]:
    print(col)
    if not os.path.exists(f"/scratch/venia/web2wiki/data/screenshots/{col}/"):
        os.mkdir(f"/scratch/venia/web2wiki/data/screenshots/{col}/")
    sc = Screenshotter(save_dir = f"/scratch/venia/web2wiki/data/screenshots/{col}/")
    temp = df2[df2[col] == 1]
    temp = temp.drop_duplicates(subset = "url")
    if len(temp)> 100:
        temp = temp.sample(100)
    url_links = temp["url"].values
    wiki_links = temp["wiki_links"].values
    sc.iterate_over(url_links, wiki_links,names)
    

is_footer


0it [00:00, ?it/s]

0


1it [00:12, 12.81s/it]

1


...

80it [14:20,  9.10s/it]

80


82it [14:30,  6.48s/it]

81
Didn't find element
82


83it [14:39, 10.60s/it]


In [ ]:
def process_image2(im_file, row, save_dir):
    IMG_DIR = "/scratch/venia/web2wiki/data/screenshots/"
    image = Image.open(save_dir+im_file + ".png")
    image = ImageOps.expand(image, border=50, fill=(255,255,255))

    draw = ImageDraw.Draw(image) 

    # specified font size
    font = ImageFont.truetype('/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf', 14) 

    text = f'url: {row["url"][0:80]}            \n Wiki link: {row["wiki_links"]}             is_wiki: {row["is_wiki"]}\
                     is_blog: {row["is_blog"]}         num_pages: {row["url_wiki_count"]}'

    # drawing text size
    draw.text((0,0), text, font = font, align ="left",fill=(0,0,0)) 

    image.save(save_dir + im_file + "_clean.png")